## Julia code for simulation

In [1]:
## necessary packages

using Distributions
using Distances
using LinearAlgebra
using SparseArrays
using IterativeSolvers
using ProgressMeter
using JLD2
using Random
using SpecialFunctions # Matern functions
using MLBase         # cross-validation 
using Convex         # compute stacking weights
using MosekTools     # compute stacking weights

In [2]:
include("./utils.jl")

sp_stacking_K_fold_MT (generic function with 1 method)

In [3]:
# Set the parameters of the simulated data #
p = 2;      # No. covariates
β = [1.0 2.0]; #regression coeff
σ2 = 1.0; ϕ = 7.0; ν = 1.0; τ2 = 1.0; # hyperparmeters in matern

In [4]:
## Generate simulation data ##
Random.seed!(1);
N = 900;                     # No. all positions
N_ho = 100;                  # No. held out positions
ind_mod = 1:(N - N_ho);      # index of training observations
coords = rand(2, N);         # random location over unit square (2 by N)
X = vcat(fill(1.0, (1, N)), rand(Normal(), (1, N)));          # design matrix (p by N)
D = pairwise(Euclidean(), coords, dims = 2);                  # distance matrix
Cov = Symmetric(Maternlu.(UpperTriangular(D), 
        ν = ν, ϕ = ϕ, σ2 = σ2))                               # covariance matrix
z = rand(MvNormal(Cov), 1);                                   # latent process
y = (β * X)[1,:] + z[:,1] + sqrt(τ2) * rand(Normal(), N);     # response

In [5]:
using BenchmarkTools

In [6]:
## candidate values of hyperparameters for stacking ##
deltasq_grid = [0.1, 0.5, 1, 2];
phi_grid = [3, 9, 15, 21];
nu_grid = [0.5, 1, 1.5, 1.75];

In [7]:
## priors parameters ##
Priors = Dict("μβ" => fill(0.0, p), "inv_V_β" => Diagonal(ones(p) * 0.25), 
    "aσ" => 2.0, "bσ" => 2.0);

In [8]:
label = "LSE"; #stacking of means
K_fold = 10;

In [9]:
label = "LP";  #stacking of predictive densities
#J = 300;       # sample size for computing posterior expectation
K_fold = 10;

In [12]:
out = sp_stacking_K_fold(X, y, coords, deltasq_grid, phi_grid,
    nu_grid, Priors, K_fold = K_fold, seed = 432, label = label, J = 300);
[out[:grid_all][(out[:w].>0.01), :] out[:w][(out[:w].>0.01)]]

Computing initial pass... 100%|██████████████████████████████████████████████████| Time: 0:00:13


2×4 Matrix{Float64}:
 3.0  0.5  1.0  0.764149
 3.0  1.0  2.0  0.235851

In [11]:
out = sp_stacking_K_fold_MT(X, y, coords, deltasq_grid, phi_grid,
    nu_grid, Priors, K_fold = K_fold, seed = 432, label = label, J = 0);
[out[:grid_all][(out[:w].>0.01), :] out[:w][(out[:w].>0.01)]]

2×4 Matrix{Float64}:
 3.0  0.5  1.0  0.764149
 3.0  1.0  2.0  0.235851

In [16]:
out = sp_stacking_K_fold_MT(X, y, coords, deltasq_grid, phi_grid,
    nu_grid, Priors, K_fold = K_fold, seed = 432, label = "LSE", J = 0);
[out[:grid_all][(out[:w].>0.01), :] out[:w][(out[:w].>0.01)]]

4×4 Matrix{Float64}:
  3.0  0.5   0.1  0.176188
  9.0  0.5   0.1  0.0991032
 21.0  1.75  0.1  0.0567573
  3.0  0.5   0.5  0.667952

In [22]:
#@benchmark sp_stacking_K_fold(X, y, coords, deltasq_grid, phi_grid,
#    nu_grid, Priors, K_fold = K_fold, seed = 3214, label = label)

In [23]:
#@benchmark sp_stacking_K_fold_MT(X, y, coords, deltasq_grid, phi_grid,
#    nu_grid, Priors, K_fold = K_fold, seed = 3214, label = label)

In [24]:
#@benchmark sp_stacking_K_fold_MT(X, y, coords, deltasq_grid, phi_grid,
#    nu_grid, Priors, K_fold = K_fold, seed = 3214, label = "LSE")

In [ ]:
## Conj_predict

In [156]:
# pre-computation and pre-allocation #
Random.seed!(3214);

K_fold = 10;
N = size(X, 2);
CV_ind_ls = collect(Kfold(N, K_fold)); # index of train data in CV
CV_ind_hold_ls = [setdiff(1:N, CV_ind_ls[k]) for k in 1:K_fold]; # index of held-out data in CV
N_grid = length(deltasq_grid) * length(phi_grid) * length(nu_grid);
nk_list = [length(x) for x in CV_ind_ls]; # be careful, different from the nk_list in my R code
nk_k_list = [(N - x) for x in nk_list];   # This is the nk_list in my R code


if X == Nothing()
    p = 0;
else
    p = size(X, 1);
    Priors["inv_V_μ_β"] = Priors["inv_V_β"] * Priors["μβ"];
    XTX = X * X'; XTy = X * y;
    XTX_list = [XTX - X[:, CV_ind_hold_ls[k]] * X[:, CV_ind_hold_ls[k]]' for k in 1:K_fold];
    XTy_list = [XTy - X[:, CV_ind_hold_ls[k]] * y[CV_ind_hold_ls[k]] for k in 1:K_fold];
end

if label == "LSE"
    y_expect = Array{Float64, 2}(undef, N, N_grid);
elseif label == "LP"
    lp_expect = Array{Float64, 2}(undef, N, N_grid);
    y_sq_sum_list = [norm(y[CV_ind_ls[k]])^2 for k in 1:K_fold];
else 
    print("label has to be LSE or LP");
end

grid_phi_nu = vcat([[x y] for x in phi_grid, y in nu_grid]...);
grid_all = vcat([[x y z] for x in phi_grid, y in nu_grid, z in deltasq_grid]...);
L_grid_deltasq  = length(deltasq_grid);

In [163]:
seed2 = 6; J = 300
## Compute expectation for stacking ##
prog = Progress(size(grid_phi_nu, 1), 1, "Computing initial pass...", 50)
Threads.@threads for i1 in 1:size(grid_phi_nu, 1)
    phi_pick = grid_phi_nu[i1, 1];
    nu_pick = grid_phi_nu[i1, 2];
    Threads.@threads for k in 1:K_fold
        Random.seed!(seed2 + (i1 - 1) * K_fold + k);
        if label == "LSE"
            y_expect[CV_ind_hold_ls[k], 
                (i1 - 1) * L_grid_deltasq .+ (1:L_grid_deltasq)] = 
            stacking_prediction_LSE(coords, nu_pick, phi_pick, deltasq_grid, 
                L_grid_deltasq, k, CV_ind_ls, CV_ind_hold_ls, p, 
                nk_list, nk_k_list, y, X, XTX_list, XTy_list, Priors);
        else
            lp_expect[CV_ind_hold_ls[k], 
                (i1 - 1) * L_grid_deltasq .+ (1:L_grid_deltasq)] = 
            stacking_prediction_LP(coords, nu_pick, phi_pick, deltasq_grid, 
                L_grid_deltasq, k, CV_ind_ls, CV_ind_hold_ls, p, 
                nk_list, nk_k_list, y, X, XTX_list, XTy_list, 
                y_sq_sum_list, Priors, J);
        end     
    end
    next!(prog)
end

Computing initial pass... 100%|██████████████████████████████████████████████████| Time: 0:00:03


In [21]:
if label == "LSE"
    w = QP_stacking_weight(y_expect, y);
else
    w = stacking_weight(lp_expect);
end
round.(reshape(w, 8, 8), digits = 3)

MOSEK warning 705: #1 (nearly) zero elements are specified in sparse row ''(383) of matrix 'A'.


8×8 Matrix{Float64}:
 -0.0  -0.0   0.0    -0.0  -0.0  -0.0  -0.0   -0.0
 -0.0  -0.0  -0.0     0.0  -0.0   0.0  -0.0    0.201
 -0.0  -0.0  -0.0     0.0  -0.0   0.0  -0.0    0.0
 -0.0   0.0  -0.0     0.0  -0.0   0.0  -0.0    0.0
 -0.0  -0.0  -0.0     0.1  -0.0   0.0  -0.0   -0.0
 -0.0  -0.0   0.0    -0.0   0.0   0.0   0.13   0.085
 -0.0  -0.0   0.483   0.0   0.0   0.0  -0.0    0.0
  0.0  -0.0  -0.0     0.0  -0.0   0.0  -0.0    0.0

In [1]:
Threads.nthreads()

8

In [7]:
versioninfo()

Julia Version 1.6.6
Commit b8708f954a (2022-03-28 07:17 UTC)
Platform Info:
  OS: Windows (x86_64-w64-mingw32)
  CPU: Intel(R) Core(TM) i7-7700K CPU @ 4.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, skylake)
Environment:
  JULIA_NUM_THREADS = 8
